# Computing LCH with Sympy

To use this notebook, execute the next block defining the differential operator function. Next, adapt the data in the following example blocks to your cases of interest.

In [14]:
import sympy as sp
from sage.all import *

def alg_mul(*args):
    result = args[0]
    for arg in args[1:]:
        result = result._mul_(arg)
    return result

def differential_operator(expr, generator_images, generator_degrees, algebra=None):
    if expr == 0:
        return 0
    if algebra:
        if len(expr.terms())>1:
            sum  = 0
            for term in expr.terms():
                sum += differential_operator(term, generator_images, generator_degrees, algebra=algebra)
            return sum
        elif len(str(expr.support()[0]).split('*'))>1:
 
            path = str(expr.support()[0]).split('*')
 
            coeff = expr.coefficients()[0]
            algebra_path = []
            for vertex in path:
                for gen in algebra.gens():
                    if vertex in str(gen):
                        algebra_path.append(gen)
            # Apply the derivation relation: d(a * b) = d(a) * b + a * d(b)
   
                        
            first_term = coeff*differential_operator(algebra_path[0], generator_images, generator_degrees, algebra=algebra) * alg_mul(*algebra_path[1:])
            degree_of_first = generator_degrees.get(algebra_path[0], 0)

            second_term = (-1)**degree_of_first *coeff* algebra_path[0] * differential_operator(alg_mul(*algebra_path[1:]), generator_images, generator_degrees, algebra=algebra)
            return first_term + second_term
        else:
            coeff = expr.coefficients()[0]
            if 1/coeff * expr in generator_images:
                return coeff * generator_images[1/coeff * expr]
            else:
                return 0
    else:
        expr = expr.expand()
        if expr in generator_images:
            return generator_images[expr]
        elif expr.is_Pow:
            base, exp = expr.as_base_exp()
            if base in generator_degrees:
                # Apply the chain rule: d(a^n) = n * a^(n-1) * d(a)
                return exp * base**(exp - 1) * differential_operator(base, generator_images, generator_degrees)
            else:
                return 0
        elif expr.is_Add:
            return sp.Add(*[differential_operator(arg, generator_images, generator_degrees) for arg in expr.args])
        elif expr.is_Mul:
            # Apply the derivation relation: d(a * b) = d(a) * b + a * d(b)
            coeff, rest = expr.as_coeff_mul()
            first_term = differential_operator(rest[0], generator_images, generator_degrees) * sp.Mul(*rest[1:]) * coeff
            degree_of_first = generator_degrees.get(rest[0], 0)
            second_term = (-1)**degree_of_first * rest[0] * differential_operator(sp.Mul(*rest[1:]), generator_images, generator_degrees) * coeff
            return first_term + second_term
        else:
            return 0



In [11]:
# This is for P5

P = DiGraph({Integer(0): {Integer(0): 'a', Integer(2): 'y'}, Integer(1): {Integer(1): 'b', Integer(2): 'w'}, Integer(2): {Integer(2): 'c', Integer(0): 'x', Integer(1): 'z'}})
# Define your quiver (documentation at https://doc.sagemath.org/html/en/reference/graphs/sage/graphs/digraph.html)


P_group = P.path_semigroup()
A = P_group.algebra(CC)

e_0, e_1, e_2, a,b,c,w,x,y,z = A.gens()
# Give a name to the generators of the algebra. Preferably, use the same names as the edges of the quiver.
# Run A.gens() in a cell to see the order and names of the edges corresponding to the generators.
# Todo - make this automatic


# Define here how the differential should act on generators.
d_vals = {
    e_0: 0,
    e_1: 0,
    e_2: 0, 
    a: -y*x,
    b: -w*z,
    c: -x*y*z*w,
    x: 0,
    y: 0,
    z: 0,
    w: 0
}


# Define here the degrees of the generators.
degrees = {
    e_0: 0,
    e_1: 0,
    e_2: 0,
    a: 1,
    b: 1,
    c: 1,
    x: 0,
    y: 0,
    z: 0,
    w: 0
}

def d(expr):
    return differential_operator(expr, d_vals, degrees, algebra = A)

# After running this cell, you can use the function d to compute the differential of any element of the algebra.
# For example, to compute the differential of a*b, run d(a*b).

In [18]:
expr = x*a
print(expr)
print(d(x*a))
print(d(expr))

1.00000000000000*x*a
-1.00000000000000*x*y*x
-1.00000000000000*x*y*x


In [8]:
# For P1 according to CM

P = DiGraph({Integer(0): {Integer(0): ['a','b','x','y','z']}})

P_group = P.path_semigroup()
A = P_group.algebra(CC)

e_0, a,b,x,y,z = A.gens()

# a,b = sp.symbols('a, b', commutative=False) # degree 1
# x,y,z = sp.symbols('x, y, z', commutative=False) # degree 0

d_vals = {
    e_0: 0,
    a: 1+x+z+x*y*z,
    b: 1+x+z+z*y*x,
    x: 0,
    y: 0,
    z: 0
    }

degrees = {
    e_0: 0,
    a: 1,
    b: 1,
    x: 0,
    y: 0,
    z: 0
}

def d(expr):
    return differential_operator(expr, d_vals, degrees, algebra=A)





In [20]:
expr = x+y+ a**2
expr = x*y*x*a*a
print(expr)
print(d(expr))


1.00000000000000*x*y*x*a*a
-1.00000000000000*x*y*x*a*z - 1.00000000000000*x*y*x*a*x + 1.00000000000000*x*y*x*z*a + 1.00000000000000*x*y*x*x*a - 1.00000000000000*x*y*x*a*x*y*z + 1.00000000000000*x*y*x*x*y*z*a


In [9]:
expr = (1+z*y)*a-b*(1+y*z)
print(d((1+z*y)*a-b*(1+y*z)))

print(d(expr))


-1.00000000000000*y*z + 1.00000000000000*z*y
-1.00000000000000*y*z + 1.00000000000000*z*y


In [144]:
factors = str(xp).split('*')
print(factors)

if 'x' in str(x):
    print('allahu akhbar')
    print(str(x))

['x', 'y', 'x', 'a', 'a']
allahu akhbar
1.00000000000000*x


In [150]:
print(x.get_custom_name())

listy = [x,y,z]
product = x.__mul_(y)
print(product)

None
1.00000000000000*x*y


In [115]:
print(2*x)
print((2*x).)


2.00000000000000*x


AttributeError: 'PathAlgebra_with_category' object has no attribute 'is_commutative'

In [60]:
def differential_operator(expr, generator_images, generator_degrees, algebra=None):
    if algebra:
    # expr = expr.expand()
        # elif expr.is_Pow:
        #     base, exp = expr.as_base_exp()
        #     if base in generator_degrees:
        #         # Apply the chain rule: d(a^n) = n * a^(n-1) * d(a)
        #         return exp * base**(exp - 1) * differential_operator(base, generator_images, generator_degrees)
        #     else:
        #         return 0
        if len(expr.terms())>1:
            sum  = 0
            for term in expr.terms():
                sum += differential_operator(term, generator_images, generator_degrees, algebra=algebra)
            return sum
        elif len(str(expr.support()[0]).split('*'))>1:
 
            path = str(expr.support()[0]).split('*')
 
            coeff = expr.coefficients()[0]
            algebra_path = []
            for vertex in path:
                for gen in algebra.gens():
                    if vertex in str(gen):
                        algebra_path.append(gen)
            # Apply the derivation relation: d(a * b) = d(a) * b + a * d(b)
   
                        
            first_term = coeff*differential_operator(algebra_path[0], generator_images, generator_degrees, algebra=algebra) * alg_mul(*algebra_path[1:])
            degree_of_first = generator_degrees.get(algebra_path[0], 0)

            second_term = (-1)**degree_of_first *coeff* algebra_path[0] * differential_operator(alg_mul(*algebra_path[1:]), generator_images, generator_degrees, algebra=algebra)
            return first_term + second_term
        else:
            coeff = expr.coefficients()[0]
            if 1/coeff * expr in generator_images:
                return coeff * generator_images[1/coeff * expr]
            else:
                return 0